## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [3]:
import re
import json
import numpy as np

In [366]:
PREPR_EXPR = re.compile(r'[^\wа-яА-Я\s\.,;:№\-/]')


def doc_preprocess(doc):
    doc = '\n' + doc.lower() + '\n'
    doc = re.sub("ппп", "пп", doc)
    doc = re.sub("- ", "", doc)
    doc = re.sub(" -", "", doc)
    doc = re.sub(r"[пнл]оста[нв]ов[пнл]е[нпл]и[ек]", r"постановление", doc)
    doc = re.sub(r"_+", "", doc)
    return PREPR_EXPR.sub('', doc)

In [367]:
F_Z = re.compile(r"\n№ ?([\wа-я\-/]+)\s")
Z = re.compile(r"\s№\s?([\wа-я\-/]+)\s")
R = re.compile(r"распоряжение\n([^\n\d]*\n){0,3}[^\n\d]*\d?\d[ \.]([а-я]+|\d\d)[ \.]\d\d\d\d[^\n]*[\s№]([\wа-я\-/]+):?\n")
P = re.compile(r"постановление\s*\n([^\n\d]*\n){0,3}[^\n\d]*\d?\d[ \.]([а-я]+|\d\d)[ \.]\d\d\d\d[^\n]*([\s№]|№ .?)([\wа-я\-/]+).?\n")

In [426]:
def extract_number(doc, doc_type, doc_date):
    doc = doc_preprocess(doc)
    
    if doc_type == "распоряжение":
        match = R.search(doc)
        if match:
            number = match.group(3)
            if re.search(r"\d", number):
                return number
    
    if doc_type == "федеральный закон" or doc_type == "распоряжение":
        matches = F_Z.findall(doc)
        if matches:
            return matches[-1]
        
    if doc_type == "закон":
        if doc.find('-кз') != -1 or doc.find('-оз') != -1 or doc.find('-рз') != -1:
            matches = re.findall("(\d+\-[кор]з)", doc)
            if matches:
                return matches[-1]
        matches = Z.findall(doc)
        if matches:
            return matches[-1]
        
    if doc_type == "постановление":
        match = P.search(doc)
        if match:
            number = match.group(4)
            if re.search(r'\b2\d\d\d', number):
                number = number[1:]
            if re.search(r"\d", number):
                return number
        matches = re.findall(r"\n№\s?([\wа-я\-/]+)\s", doc)
        if matches:
            return matches[-1]
        
    if doc_type == "указ":
        matches = re.findall(r"\n№\s(\d+)\s", doc)
        if matches:
            return matches[-1]
    
    match = re.search(r"№\s?([\wа-я\-/]+)\s", doc)
    if match:
        return match.group(1)
    return ""

In [427]:
def predict(test, types, dates):
    results = []
    for doc, doc_type, doc_date in zip(test, types, dates):
        number = extract_number(doc, doc_type, doc_date)
        prediction = {"type": "",
                      "date": "",
                      "number": number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [428]:
from eval_module import quality

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
true_dates = np.array([x['date'] for x in labeled])
predicted = predict(train, true_types, true_dates)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.9726748559,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [429]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type} ({ids[0].shape[0]}): {quality(predicted_labels[ids], test_labels[ids])['number_accuracy']}")

федеральный закон (192): 1.0
постановление (2459): 0.9829198861
приказ (77): 0.6883116883
распоряжение (430): 0.9906976744
закон (199): 0.959798995
указ (632): 0.9509493671


In [430]:
doc_type = 'приказ'
ids = np.where(true_types == doc_type)

y_test = [(x['number'], x['type'], x['date']) for x in test_labels[ids]]
y_pred = [x['number'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("=====")

right value = ('174', 'приказ', '26.10.2015')
wrong value = 5я
docname = train/txts/20549ec7255983ebbb47d6f42eb0ea02194f863a.txt
МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ
РОССИИСКОМ ФЕДЕРАНИИ

 

ПО ДЕЛАМ МОЛОДЕЖИ `
(%ОСВ/[ОЛОДЕ){%Ь) \ ЗАРЕГИСТРИРОВАНО

| Регистрационный № 5Я 8о
ПРИКАЗ | - от БУонЕ а0

26 октября 2015 г. 174

‚ №

› Москва

Об утверждении Положения о комиссии Федерального агентства по делам
молодежи по соблюдению требований к служебному (должностному) поведению
федеральных государственных служащих и работников организаций,
созданных для выполнения задач, поставленных перед Федеральным
агентством по делам молодежи, и урегулированию конфликта интересов

В соответствии с федеральными законами от 27 июля 2004 г. № 79-ФЗ
"О государственной гражданской службе Российской Федерации" (Собрание
законодательства Российской Федерации, 2004, № 31, ст. 3215; 2006, № 6, ст. 636;
2007, № 10, ст. 1151; № 16, ст. 1828; № 49, ст. 6070; 2008, № 13, ст. 1186; № 30 (ч. 2),
ст. 3616; № 52 (ч. 1), ст.